# Amplitude Modulation Variants

In [ ]:
from math import sqrt
from disiple.signals import AudioSignal, PowerSpectrum
from disiple.util import nextpow2
import numpy as np
from bokeh.plotting import show
from bokeh.layouts import row, column, gridplot
from bokeh.models import CustomJS, Slider, Div

In [ ]:
samplerate = 22050
duration = nextpow2(samplerate) / samplerate # at least 1 second
times = np.arange(0, duration, 1/samplerate)

### Message Signal

In [ ]:
amp_mod = 0.3
freq_mod = 220/duration
samples_mod = amp_mod * np.sin(2*np.pi*freq_mod*times)
mod_signal = AudioSignal(samples_mod, samplerate)

### Carrier Signal

In [ ]:
amp_carr = 0.5
freq_carr = 2000
samples_carr = amp_carr * np.sin(2*np.pi*freq_carr*times)
carr_signal = AudioSignal(samples_carr, samplerate)

### Amplitude Modulation

In [ ]:
mod_idx = amp_mod / amp_carr
samples_am = (1 + samples_mod / amp_carr) * samples_carr
am_signal = AudioSignal(samples_am, samplerate)
am_pspec = PowerSpectrum.from_timesignal(am_signal, dB=True)

In [ ]:
amp_dssc = sqrt(amp_carr**2/2 + amp_mod**2/4)
samples_dssc = amp_dssc * np.cos(2*np.pi*(freq_carr-freq_mod)*times) - amp_dssc * np.cos(2*np.pi*(freq_carr+freq_mod)*times)
dssc_signal = AudioSignal(samples_dssc, samplerate)
dssc_pspec = PowerSpectrum.from_timesignal(dssc_signal, dB=True)

In [ ]:
amp_ssb = sqrt(amp_carr**2 + amp_mod**2/2)
samples_ssb = -amp_ssb * np.cos(2*np.pi*(freq_carr+freq_mod)*times)
ssb_signal = AudioSignal(samples_ssb, samplerate)
ssb_pspec = PowerSpectrum.from_timesignal(ssb_signal, dB=True)

### Create Figures

In [ ]:
time_range = (0, 6/freq_mod) # show first 6 periods of signal = 6 * 1/freq_mod
dB_range = (-60, -10)
min_freq_mod, max_freq_mod = freq_mod / 2, 2 * freq_mod
freq_range = (freq_carr-max_freq_mod-5, freq_carr+max_freq_mod+5)

am_fig = am_signal.plot(title='Double Side-Band Signal', x_range=time_range, active_inspect=None, line_color='olive')
dssc_fig = dssc_signal.plot(title='Double Side-Band Suppressed Carrier Signal', x_range=time_range, active_inspect=None, line_color='purple')
ssb_fig = ssb_signal.plot(title='Single Side-Band Signal', x_range=time_range, active_inspect=None, line_color='crimson')

am_pspec_fig = am_pspec.plot(title='Power Spectrum of DSB Signal', x_range=freq_range, y_range=dB_range, active_inspect=None, line_color='olive')
dssc_pspec_fig = dssc_pspec.plot(title='Power Spectrum of DSB-SC Signal', x_range=freq_range, y_range=dB_range, active_inspect=None, line_color='purple')
ssb_pspec_fig = ssb_pspec.plot(title='Power Spectrum of SSB Signal', x_range=freq_range, y_range=dB_range, active_inspect=None, line_color='crimson')

mod_idx_div = Div(text=f'Modulation Factor: <b>{100*mod_idx:.0f}%</b>', width=80, height=10)

### Link time and frequency axes

In [ ]:
from itertools import product
def link_x_axes(figs):
    for fig1, fig2 in product(figs, figs):
        fig1.x_range.js_link('start', fig2.x_range, 'start')
        fig1.x_range.js_link('end', fig2.x_range, 'end')
link_x_axes({am_fig, dssc_fig, ssb_fig})
link_x_axes({am_pspec_fig, dssc_pspec_fig, ssb_pspec_fig})

### Display figures

In [ ]:
plot = row(
    gridplot([am_fig, dssc_fig, ssb_fig], ncols=1, width=600),
    gridplot([am_pspec_fig, dssc_pspec_fig, ssb_pspec_fig], ncols=1, width=600),
)
show(plot)

### Add Interaction

In [ ]:
amp_mod_slider = Slider(start=0, end=0.5, value=amp_mod, step=.01, title='Message Amplitude')
freq_mod_slider = Slider(start=min_freq_mod, end=max_freq_mod, value=freq_mod, step=10, title='Message Frequency')
amp_carr_slider = Slider(start=0, end=0.5, value=amp_carr, step=.01, title='Carrier Amplitude')
freq_carr_slider = Slider(start=1000, end=min(4000, samplerate/2-max_freq_mod), value=freq_carr, step=100, title='Carrier Frequency')

callback = CustomJS(args=dict(amSource=am_fig.renderers[0].data_source,
                              dsscSource=dssc_fig.renderers[0].data_source,
                              ssbSource=ssb_fig.renderers[0].data_source,
                              amPspecSource=am_pspec_fig.renderers[0].data_source,
                              dsscPspecSource=dssc_pspec_fig.renderers[0].data_source,
                              ssbPspecSource=ssb_pspec_fig.renderers[0].data_source,
                              samplerate=samplerate,
                              ampModSlider=amp_mod_slider,
                              freqModSlider=freq_mod_slider,
                              ampCarrSlider=amp_carr_slider,
                              freqCarrSlider=freq_carr_slider,
                              modIdxDiv=mod_idx_div), code="""
    const ampMod = ampModSlider.value;
    const ampCarr = ampCarrSlider.value;
    const freqMod = freqModSlider.value;
    const freqCarr = freqCarrSlider.value;
    const modIdx = ampMod / ampCarr;

    const amSamples = amSource.data.x.map((t) => Math.max(0, 1 + modIdx * Math.sin(2*Math.PI*freqMod*t)) * ampCarr * Math.sin(2*Math.PI*freqCarr*t));
    amSource.data = {'x': amSource.data.x, 'y': amSamples};
    const ampDssc = Math.sqrt(Math.pow(ampCarr, 2) / 2 + Math.pow(ampMod, 2) / 4);
    const dsscSamples = dsscSource.data.x.map((t) => ampDssc * Math.cos(2*Math.PI*(freqCarr-freqMod)*t) - ampDssc * Math.cos(2*Math.PI*(freqCarr+freqMod)*t));
    dsscSource.data = {'x': dsscSource.data.x, 'y': dsscSamples};
    const ampSsb = Math.sqrt(Math.pow(ampCarr, 2) + Math.pow(ampMod, 2) / 2);
    const ssbSamples = ssbSource.data.x.map((t) => -ampSsb * Math.cos(2*Math.PI*(freqCarr+freqMod)*t));
    ssbSource.data = {'x': ssbSource.data.x, 'y': ssbSamples};
    const modFactor = 100 * modIdx;
    const style = modIdx > 1 ? 'style="color:red"' : '';
    modIdxDiv.text = `Modulation Factor: <b ${style}>${modFactor.toFixed(0)}%</b>`;

    const freqs = fourier.rfftFreqs(samplerate);
    amPspecSource.data = {'bin_unit': freqs, 'frequency': freqs, 'magnitude': fourier.dBSpectrum(amSamples)};
    dsscPspecSource.data = {'bin_unit': freqs, 'frequency': freqs, 'magnitude': fourier.dBSpectrum(dsscSamples)};
    ssbPspecSource.data = {'bin_unit': freqs, 'frequency': freqs, 'magnitude': fourier.dBSpectrum(ssbSamples)};
""")
amp_mod_slider.js_on_change('value', callback)
freq_mod_slider.js_on_change('value', callback)
amp_carr_slider.js_on_change('value', callback)
freq_carr_slider.js_on_change('value', callback)

freq_carr_slider.js_on_change('value', CustomJS(args=dict(
    amPspecRange=am_pspec_fig.x_range, dsscPspecRange=dssc_pspec_fig.x_range,
    ssbPspecRange=ssb_pspec_fig.x_range, maxFreqMod=max_freq_mod), code="""
    amPspecRange.start = cb_obj.value - maxFreqMod - 5;
    amPspecRange.end = cb_obj.value + maxFreqMod + 5;
    dsscPspecRange.start = cb_obj.value - maxFreqMod - 5;
    dsscPspecRange.end = cb_obj.value + maxFreqMod + 5;
    ssbPspecRange.start = cb_obj.value - maxFreqMod - 5;
    ssbPspecRange.end = cb_obj.value + maxFreqMod + 5;
"""))

interative_plot = column(
    row(amp_mod_slider, freq_mod_slider, amp_carr_slider, freq_carr_slider),
    mod_idx_div,
    row(
        gridplot([am_fig, dssc_fig, ssb_fig], ncols=1, width=600),
        gridplot([am_pspec_fig, dssc_pspec_fig, ssb_pspec_fig], ncols=1, width=600),
    )
)
show(interative_plot)

In [ ]:
from bokeh.plotting import save
from bokeh.resources import INLINE
template = f"""
{{% block postamble %}}
    <script src="fourier.js"></script>
    <script>const fourier = new Fourier({len(times)});</script>
{{% endblock %}} """
save(interative_plot, filename='am-variants.html', title='Amplitude Modulation Variants', resources=INLINE, template=template)